# Julia is fast

Very often, benchmarks are used to compare languages.  These benchmarks can lead to long discussions, first as to exactly what is being benchmarked and secondly what explains the differences.  These simple questions can sometimes get more complicated than you at first might imagine.

The purpose of this notebook is for you to see a simple benchmark for yourself.  One can read the notebook and see what happened on the author's Macbook Pro with a 4-core Intel Core I7, or run the notebook yourself.

(This material began life as a wonderful lecture by Steven Johnson at MIT: https://github.com/stevengj/18S096-iap17/blob/master/lecture1/Boxes-and-registers.ipynb.)

# `sum`: An easy enough function to understand

Consider the  **sum** function `sum(a)`, which computes
$$
\mathrm{sum}(a) = \sum_{i=1}^n a_i,
$$
where $n$ is the length of `a`.

In [1]:
a = rand(10^7) # 1D vector of random numbers, uniform on [0,1)

10000000-element Array{Float64,1}:
 0.19224  
 0.871065 
 0.44443  
 0.565001 
 0.109785 
 0.0121026
 0.94172  
 0.571784 
 0.270109 
 0.693784 
 0.838322 
 0.468047 
 0.992995 
 ⋮        
 0.0953679
 0.538208 
 0.947862 
 0.455662 
 0.660807 
 0.450632 
 0.322782 
 0.458957 
 0.955901 
 0.970387 
 0.426714 
 0.40768  

In [2]:
sum(a)   

5.000631461534377e6

The expected result is 0.5 * 10^7, since the mean of each entry is 0.5

# Benchmarking a few ways in a few languages

Julia has a `BenchmarkTools.jl` package for easy and accurate benchmarking:

In [6]:
#Pkg.add("BenchmarkTools")

In [7]:
using BenchmarkTools  

INFO: Recompiling stale cache file /Users/edelman/.julia/lib/v0.6/JLD.ji for module JLD.


#  1. The C language

C is often considered the gold standard: difficult on the human, nice for the machine. Getting within a factor of 2 of C is often satisfying. Nonetheless, even within C, there are many kinds of optimizations possible that a naive C writer may or may not get the advantage of.

The current author does not speak C, so he does not read the cell below, but is happy to know that you can put C code in a Julia session, compile it, and run it. Note that the `"""` wrap a multi-line string.

In [8]:
C_code = """
#include <stddef.h>
double c_sum(size_t n, double *X) {
    double s = 0.0;
    for (size_t i = 0; i < n; ++i) {
        s += X[i];
    }
    return s;
}
"""

const Clib = tempname()   # make a temporary file


# compile to a shared library by piping C_code to gcc
# (works only if you have gcc installed):

open(`gcc -fPIC -O3 -msse3 -xc -shared -o $(Clib * "." * Libdl.dlext) -`, "w") do f
    print(f, C_code) 
end

# define a Julia function that calls the C function:
c_sum(X::Array{Float64}) = ccall(("c_sum", Clib), Float64, (Csize_t, Ptr{Float64}), length(X), X)

c_sum (generic function with 1 method)

In [9]:
c_sum(a)

5.000631461534445e6

In [10]:
c_sum(a) ≈ sum(a) # type \approx and then <TAB> to get the ≈ symbolb

true

In [11]:
≈  # alias for the `isapprox` function

isapprox (generic function with 3 methods)

In [12]:
?isapprox

search: isapprox



```
isapprox(x, y; rtol::Real=sqrt(eps), atol::Real=0, nans::Bool=false, norm::Function)
```

Inexact equality comparison: `true` if `norm(x-y) <= atol + rtol*max(norm(x), norm(y))`. The default `atol` is zero and the default `rtol` depends on the types of `x` and `y`. The keyword argument `nans` determines whether or not NaN values are considered equal (defaults to false).

For real or complex floating-point values, `rtol` defaults to `sqrt(eps(typeof(real(x-y))))`. This corresponds to requiring equality of about half of the significand digits. For other types, `rtol` defaults to zero.

`x` and `y` may also be arrays of numbers, in which case `norm` defaults to `vecnorm` but may be changed by passing a `norm::Function` keyword argument. (For numbers, `norm` is the same thing as `abs`.) When `x` and `y` are arrays, if `norm(x-y)` is not finite (i.e. `±Inf` or `NaN`), the comparison falls back to checking whether all elements of `x` and `y` are approximately equal component-wise.

The binary operator `≈` is equivalent to `isapprox` with the default arguments, and `x ≉ y` is equivalent to `!isapprox(x,y)`.

```jldoctest
julia> 0.1 ≈ (0.1 - 1e-10)
true

julia> isapprox(10, 11; atol = 2)
true

julia> isapprox([10.0^9, 1.0], [10.0^9, 2.0])
true
```


We can now benchmark the C code directly from Julia:

In [13]:
c_bench = @benchmark c_sum($a) 

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     8.298 ms (0.00% GC)
  median time:      8.819 ms (0.00% GC)
  mean time:        9.010 ms (0.00% GC)
  maximum time:     16.176 ms (0.00% GC)
  --------------
  samples:          554
  evals/sample:     1

In [14]:
println("C: Fastest time was $(minimum(c_bench.times) / 1e6) msec")

C: Fastest time was 8.2977 msec


In [15]:
d = Dict()  # a "dictionary", i.e. an associative array
d["C"] = minimum(c_bench.times) / 1e6  # in milliseconds
d

Dict{Any,Any} with 1 entry:
  "C" => 8.2977

In [16]:
using Plots
gr()

Plots.GRBackend()

In [17]:
t = c_bench.times / 1e6 # times in milliseconds
m, σ = minimum(t), std(t)

histogram(t, bins=500,
    xlim=(m - 0.01, m + σ),
    xlabel="milliseconds", ylabel="count", label="")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 8.50 
 
 
 8.75 
 
 
 9.00 
 
 
 0 
 
 
 5 
 
 
 10 
 
 
 15 
 
 
 20 
 
 
 milliseconds 
 
 
 count

# 2. Python's built in `sum` 

The `PyCall` package provides a Julia interface to Python:

In [20]:
#Pkg.add("PyCall")

In [21]:
using PyCall

In [22]:
# Call a low-level PyCall function to get a Python list, because
# by default PyCall will convert to a NumPy array instead (we benchmark NumPy below):

apy_list = PyCall.array2py(a, 1, 1)

# get the Python built-in "sum" function:
pysum = pybuiltin("sum")

PyObject <built-in function sum>

In [23]:
pysum(a)

5.000631461534445e6

In [24]:
pysum(a) ≈ sum(a)

true

In [25]:
py_list_bench = @benchmark $pysum($apy_list)

BenchmarkTools.Trial: 
  memory estimate:  512 bytes
  allocs estimate:  17
  --------------
  minimum time:     73.492 ms (0.00% GC)
  median time:      93.611 ms (0.00% GC)
  mean time:        92.735 ms (0.00% GC)
  maximum time:     109.941 ms (0.00% GC)
  --------------
  samples:          55
  evals/sample:     1

In [26]:
d["Python built-in"] = minimum(py_list_bench.times) / 1e6
d

Dict{Any,Any} with 2 entries:
  "C"               => 8.2977
  "Python built-in" => 73.492

# 3. Python: `numpy` 

## Takes advantage of hardware "SIMD", but only works when it works.

`numpy` is an optimized C library, callable from Python.
It may be installed within Julia as follows:

In [29]:
using Conda 
#Conda.add("numpy")

In [30]:
numpy_sum = pyimport("numpy")["sum"]
apy_numpy = PyObject(a) # converts to a numpy array by default

py_numpy_bench = @benchmark $numpy_sum($apy_numpy)

BenchmarkTools.Trial: 
  memory estimate:  720 bytes
  allocs estimate:  22
  --------------
  minimum time:     4.749 ms (0.00% GC)
  median time:      5.327 ms (0.00% GC)
  mean time:        5.598 ms (0.00% GC)
  maximum time:     10.697 ms (0.00% GC)
  --------------
  samples:          890
  evals/sample:     1

In [31]:
numpy_sum(apy_list) # python thing

5.0006314615343725e6

In [32]:
numpy_sum(apy_list) ≈ sum(a)

true

In [33]:
d["Python numpy"] = minimum(py_numpy_bench.times) / 1e6
d

Dict{Any,Any} with 3 entries:
  "C"               => 8.2977
  "Python numpy"    => 4.74906
  "Python built-in" => 73.492

# 4. Python, hand-written 

In [34]:
py"""
def py_sum(a):
    s = 0.0
    for x in a:
        s = s + x
    return s
"""

sum_py = py"py_sum"

PyObject <function py_sum at 0x14c44a758>

In [35]:
py_hand = @benchmark $sum_py($apy_list)

BenchmarkTools.Trial: 
  memory estimate:  512 bytes
  allocs estimate:  17
  --------------
  minimum time:     497.583 ms (0.00% GC)
  median time:      504.909 ms (0.00% GC)
  mean time:        526.745 ms (0.00% GC)
  maximum time:     620.353 ms (0.00% GC)
  --------------
  samples:          10
  evals/sample:     1

In [36]:
sum_py(apy_list)

5.000631461534445e6

In [37]:
sum_py(apy_list) ≈ sum(a)

true

In [38]:
d["Python hand-written"] = minimum(py_hand.times) / 1e6
d

Dict{Any,Any} with 4 entries:
  "C"                   => 8.2977
  "Python numpy"        => 4.74906
  "Python hand-written" => 497.583
  "Python built-in"     => 73.492

# 5. Julia (built-in) 

## Written directly in Julia, not in C!

In [39]:
@which sum(a)

sum(a) in Base at reduce.jl:359

In [40]:
j_bench = @benchmark sum($a)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     4.720 ms (0.00% GC)
  median time:      5.990 ms (0.00% GC)
  mean time:        6.216 ms (0.00% GC)
  maximum time:     16.094 ms (0.00% GC)
  --------------
  samples:          802
  evals/sample:     1

In [41]:
d["Julia built-in"] = minimum(j_bench.times) / 1e6
d

Dict{Any,Any} with 5 entries:
  "C"                   => 8.2977
  "Python numpy"        => 4.74906
  "Python hand-written" => 497.583
  "Python built-in"     => 73.492
  "Julia built-in"      => 4.71977

# 6. Julia (hand-written) 

In [42]:
function mysum(A)   
    s = 0.0  # s = zero(eltype(A))
    for a in A
        s += a
    end
    s
end

mysum (generic function with 1 method)

In [43]:
j_bench_hand = @benchmark mysum($a)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     8.717 ms (0.00% GC)
  median time:      9.747 ms (0.00% GC)
  mean time:        10.084 ms (0.00% GC)
  maximum time:     15.670 ms (0.00% GC)
  --------------
  samples:          495
  evals/sample:     1

In [44]:
d["Julia hand-written"] = minimum(j_bench_hand.times) / 1e6
d

Dict{Any,Any} with 6 entries:
  "C"                   => 8.2977
  "Python numpy"        => 4.74906
  "Julia hand-written"  => 8.71737
  "Python hand-written" => 497.583
  "Python built-in"     => 73.492
  "Julia built-in"      => 4.71977

# Summary

In [45]:
for (key, value) in sort(collect(d))
    println(rpad(key, 20, "."), lpad(round(value, 1), 8, "."))
end

C........................8.3
Julia built-in...........4.7
Julia hand-written.......8.7
Python built-in.........73.5
Python hand-written....497.6
Python numpy.............4.7


In [46]:
for (key, value) in sort(collect(d), by=x->x[2])
    println(rpad(key, 20, "."), lpad(round(value, 2), 10, "."))
end

Julia built-in............4.72
Python numpy..............4.75
C..........................8.3
Julia hand-written........8.72
Python built-in..........73.49
Python hand-written.....497.58
